In [1]:
#Import all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap

import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
from random import randint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

In [2]:
#Load the data - Train
train= pd.read_csv('train.csv')
#train.columns= [col.replace(' ', '_') for col in train.columns]
train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [3]:
#Load the data - Test
test= pd.read_csv('test.csv')
#test.columns= [col.replace(' ', '_') for col in test.columns]
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [ ]:
# Something to explore - note the data is quite unbalanced
# Running this did not improve my model so was not part of final submission
#from sklearn.utils import resample
#believe = train[train['sentiment'] == 1]
#no_belief = train[train['sentiment'] == -1]
#neutral = train[train['sentiment'] == 0]
#news = train[train['sentiment'] == 2]
# # upsample minority
#no_belief_upsampled = resample(no_belief,
#                         replace=True, # sample with replacement
#                         n_samples=len(believe), # match number in majority class
#                         random_state=27) # reproducable results
#neutral_upsampled = resample(neutral,
#                         replace=True, # sample with replacement
#                         n_samples=len(believe), # match number in majority class
#                         random_state=27) # reproducable results
#news_upsampled = resample(news,
#                         replace=True, # sample with replacement
#                         n_samples=len(believe), # match number in majority class
#                         random_state=27) # reproducable results
# # combine majority and upsampled minority
#upsampled = pd.concat([believe,no_belief_upsampled,neutral_upsampled,new_upsampled ])

In [ ]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [ ]:
#Cleaning the data from the messages - this made it worse
#This was not used as part of the final submission

# Remove urls
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
train['message'] = train['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)
test['message'] = test['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

# Remove punctuation
import string
def remove_punctuation_numbers(message):
    punc_numbers = string.punctuation + '0123456789'
    return ''.join([l for l in message if l not in punc_numbers])
train['message'] = train['message'].apply(remove_punctuation_numbers)
test['message'] = test['message'].apply(remove_punctuation_numbers)

# Make lower case
train['message'] = train['message'].str.lower()
test['message'] = test['message'].str.lower()

In [4]:
#select features
y = train['sentiment']
X = train['message']

In [5]:
#Encode the words
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

In [6]:
#train test split - stratify=y,shuffle=True - x_train, x_val - stratify used when you have an unblanaced data set
#X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.3,shuffle=True, stratify=y, random_state=11)
#train test split
x_train, x_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=11)

In [ ]:
#Building Classifiers
names = ['Logistic Regression', 'Nearest Neighbors', 
         'Linear SVM', 'RBF SVM',          
         'Decision Tree', 'Random Forest',  'AdaBoost',
         'Naive Bayes']

classifiers = [
    LogisticRegression(max_iter=500), 
    KNeighborsClassifier(9),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=0.1, C=10),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),    
    AdaBoostClassifier(),
    BernoulliNB()    
] 

In [ ]:
#Training the models
results = []

models = {}
confusion = {}
class_report = {}


for name, clf in zip(names, classifiers):    
    print ('Fitting {:s} model...'.format(name))
    run_time = %timeit -q -o clf.fit(x_train, y_train)
    
    print ('... predicting')
    y_pred = clf.predict(x_train)   
    y_pred_test = clf.predict(x_test)
    
    print ('... scoring')
    accuracy  = metrics.accuracy_score(y_train, y_pred)
    precision = metrics.precision_score(y_train, y_pred, average='macro')
    recall    = metrics.recall_score(y_train, y_pred, average='macro')
    
    f1        = metrics.f1_score(y_train, y_pred, average='macro')    
    f1_test   = metrics.f1_score(y_test, y_pred_test, average='macro')    
    
    # Save the results to dictionaries
    models[name] = clf    
    confusion[name] = metrics.confusion_matrix(y_train, y_pred)
    class_report[name] = metrics.classification_report(y_train, y_pred)
    
    results.append([name, accuracy, precision, recall, f1, f1_test, run_time.best])

    
results = pd.DataFrame(results, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F1 Train', 'F1 Test', 'Train Time'])
results.set_index('Classifier', inplace= True)

Fitting Logistic Regression model...
... predicting
... scoring
Fitting Nearest Neighbors model...
... predicting
... scoring
Fitting Linear SVM model...
... predicting
... scoring
Fitting RBF SVM model...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


... predicting
... scoring
Fitting Decision Tree model...
... predicting
... scoring
Fitting Random Forest model...
... predicting
... scoring
Fitting AdaBoost model...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


... predicting
... scoring
Fitting Naive Bayes model...
... predicting
... scoring


In [ ]:
#Lets see your results
results.sort_values('F1 Train', ascending=False)

,Accuracy,Precision,Recall,F1 Train,F1 Test,Train Time
Classifier,,,,,,
RBF SVM,0.922695,0.931918,0.879303,0.902954,0.602142,14.512465
Logistic Regression,0.805202,0.859538,0.647982,0.704646,0.534650,2.641638
Nearest Neighbors,0.703784,0.696257,0.612515,0.628308,0.518832,0.001355
Naive Bayes,0.729071,0.824749,0.510802,0.552655,0.479041,0.005675
AdaBoost,0.597670,0.647713,0.397632,0.431147,0.422241,3.567553
Decision Tree,0.571390,0.673088,0.327893,0.316071,0.315686,0.104683
Linear SVM,0.541949,0.135487,0.250000,0.175735,0.173814,10.340920
Random Forest,0.541949,0.135487,0.250000,0.175735,0.173814,0.032639


In [ ]:
#lets do a gridsearch to get the best hyperparameter for my best model
param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf','poly','sigmoid']}
grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)
grid.fit(x_train,y_train)
grid.best_params_

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] C=1, gamma=1, kernel=linear .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................... C=1, gamma=1, kernel=linear, total=  13.1s
[CV] C=1, gamma=1, kernel=linear .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.1s remaining:    0.0s


[CV] ...................... C=1, gamma=1, kernel=linear, total=  13.1s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=  13.2s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=  13.1s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=  13.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=  21.4s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=  21.4s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=  21.4s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] .

[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed: 89.0min finished


{'C': 10, 'gamma': 1, 'kernel': 'rbf'}

In [ ]:
#refine the gridsearch
param_grid = {'C':[1,2,3,4,5,6,7,8,9,10],'gamma':[1,0.1,0.001,0.0001]}
grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)
grid.fit(x_train,y_train)
grid.best_params_

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=1, gamma=1 ....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..................................... C=1, gamma=1, total=  24.0s
[CV] C=1, gamma=1 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.0s remaining:    0.0s


[CV] ..................................... C=1, gamma=1, total=  24.4s
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=  25.7s
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=  25.8s
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=  25.8s
[CV] C=1, gamma=0.1 ..................................................
[CV] ................................... C=1, gamma=0.1, total=  20.1s
[CV] C=1, gamma=0.1 ..................................................
[CV] ................................... C=1, gamma=0.1, total=  19.1s
[CV] C=1, gamma=0.1 ..................................................
[CV] ................................... C=1, gamma=0.1, total=  19.2s
[CV] C=1, gamma=0.1 ..................................................
[CV] .

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 72.4min finished


{'C': 8, 'gamma': 0.1}

In [ ]:
#refining further
param_grid = {'C':[7,8,9,10],'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)
grid.fit(x_train,y_train)
grid.best_params_

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=7, gamma=0.1 ..................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................... C=7, gamma=0.1, total=  18.2s
[CV] C=7, gamma=0.1 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.2s remaining:    0.0s


[CV] ................................... C=7, gamma=0.1, total=  18.1s
[CV] C=7, gamma=0.1 ..................................................
[CV] ................................... C=7, gamma=0.1, total=  18.4s
[CV] C=7, gamma=0.1 ..................................................
[CV] ................................... C=7, gamma=0.1, total=  18.3s
[CV] C=7, gamma=0.1 ..................................................
[CV] ................................... C=7, gamma=0.1, total=  18.4s
[CV] C=7, gamma=0.2 ..................................................
[CV] ................................... C=7, gamma=0.2, total=  19.5s
[CV] C=7, gamma=0.2 ..................................................
[CV] ................................... C=7, gamma=0.2, total=  19.5s
[CV] C=7, gamma=0.2 ..................................................
[CV] ................................... C=7, gamma=0.2, total=  20.3s
[CV] C=7, gamma=0.2 ..................................................
[CV] .

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 86.6min finished


{'C': 8, 'gamma': 0.9}

In [7]:
#using the best model from above and after performing the grid search to get the best parameters
#svc = SVC(gamma=0.1, C=10) 
svc = SVC(gamma=0.1, C=8) ## this is the best submitted
#svc = SVC(gamma=0.9, C=10)
svc.fit(x_train,y_train)
svc_pred = svc.predict(x_test)

In [ ]:
#test a different model - this did not give a good result
#other models were also tested in other notebooks but did not improve overall result
#from sklearn.ensemble import GradientBoostingClassifier
#gb_clf = GradientBoostingClassifier(n_estimators=20, max_features=2, max_depth=2, random_state=0)
#gb_clf.fit(x_train, y_train)
#predictions = gb_clf.predict(x_test)

In [8]:
#make predictions
f1_score(y_test, svc_pred, average="macro")
#f1_score(y_test, predictions, average="macro")

0.6687153335471333

In [9]:
#get the test set ready 
testx = test['message']
test_vect = vectorizer.transform(testx)

In [10]:
#make predictions
y_pred = svc.predict(test_vect)

In [11]:
test['sentiment'] = y_pred

In [12]:
#test.head()
len(test)

10546

In [13]:
#create submission
test[['tweetid','sentiment']].to_csv('Michelle_Bosch_FINAL_Submission_V6_2.csv', index=False)